In [1]:
import os
import sys
import glob
import time
import copy
import logging
import random
import numpy as np
import torch
import torch.nn as nn
import torch.utils
import torch.nn.functional as F
import model_utils.configure as conf
from model import Graph2Seq
import utils

import math
import json
from collections import OrderedDict

import numpy as np
import torch
import torch.utils.data
import torch.nn.functional as F

INPUT = 'input'
CONV1X1 = 'conv1x1-bn-relu'
CONV3X3 = 'conv3x3-bn-relu'
MAXPOOL3X3 = 'maxpool3x3'
OUTPUT = 'output'

mode = "train"
random.seed(conf.seed)
np.random.seed(conf.seed)
torch.manual_seed(conf.seed)
logging.info("conf = %s", conf)

conf.source_length = conf.encoder_length = conf.decoder_length = (conf.nodes + 2) * (conf.nodes - 1) // 2
epochs = conf.epochs

In [25]:
from importlib import reload
reload(model_utils.aggregators)

NameError: name 'model_utils' is not defined

In [2]:
import math
import json
from collections import OrderedDict

import numpy as np
import torch
import torch.utils.data
import torch.nn.functional as F
import model_utils.configure as conf
"""
0: sos/eos
1: no connection
2: connection
3: CONV1X1
4: CONV3X3
5: MAXPOOL3X3
6: OUTPUT
"""
        
class ControllerDataset(torch.utils.data.Dataset):
    def __init__(self, file_path):
        super(ControllerDataset, self).__init__()

        self.adjacency_matrices = []
        self.operations = []
        self.sequences = []

        with open(file_path, "r") as f:
            for line in f.readlines():
                line = line.strip()

                jo = json.loads(line, object_pairs_hook=OrderedDict)

                self.adjacency_matrices.append(jo['module_adjacency'])
                self.operations.append(jo['module_operations'])
                self.sequences.append(jo['sequence'])

    def __getitem__(self, index):
        operations = self.operations[index]
        num_nodes = len(operations)

        ops = []
        for op in operations:
            if op == CONV1X1:
                ops.append(3)
            elif op == CONV3X3:
                ops.append(4)
            elif op == MAXPOOL3X3:
                ops.append(5)
            elif op == OUTPUT:
                ops.append(6)
            if op == INPUT:
                ops.append(7)

        sample = {
            'matrix' : torch.LongTensor(self.adjacency_matrices[index]),
            'operations': torch.LongTensor(ops),
            'sequence': torch.LongTensor(self.sequences[index])
        }
        return sample
    
    def __len__(self):
        return len(self.sequences)

In [8]:
def collate_fn(samples):
    ## transform into batch samples

    ## create node -> global idnex
    ## global index -> neighbor's global index

    degree_max_size = conf.degree_max_size
    graph_size = conf.graph_size
    # degree_max_size = 5
    # graph_size = 7
    seq_max_length = int((graph_size+2)*(graph_size-1)/2)

    g_idxs = []
    g_fw_adjs = []
    g_bw_adjs = []
    g_operations = []
    g_sequence = []
    g_num_nodes = []

    g_idx_base = 0
    for g_idx, sample in enumerate(samples):
        matrix = sample['matrix']
        num_nodes = matrix.shape[0]
        g_num_nodes.append(num_nodes)

        for row in range(num_nodes):
            g_fw_adjs.append(list())
            g_bw_adjs.append(list())

        for row in range(num_nodes):
           for col in range(row+1, num_nodes):
            if matrix[row][col] :
                g_fw_adjs[g_idx_base + row].append(g_idx_base + col)
                g_bw_adjs[g_idx_base + col].append(g_idx_base + row)

        for op in sample['operations']:
            g_operations.append(op)

        sequence = sample['sequence']

        sequence = torch.cat([sequence, torch.LongTensor([0] * (seq_max_length - len(sequence)))])
        g_sequence.append(sequence)

        g_idx_base += num_nodes

    for idx in range(len(g_fw_adjs)):
        g_fw_adjs[idx].extend([g_idx_base] * (degree_max_size - len(g_fw_adjs[idx])))
        g_bw_adjs[idx].extend([g_idx_base] * (degree_max_size - len(g_bw_adjs[idx])))
        
    g_operations.append(0)

    g_num_nodes = torch.LongTensor(g_num_nodes)

    # [batch_size, conf.degree_max_size]
    g_fw_adjs = torch.LongTensor(g_fw_adjs)
    g_bw_adjs = torch.LongTensor(g_bw_adjs)

    # [batch_size +1] # due to padding
    g_operations = torch.LongTensor(g_operations)

    # [sum of sequence_length]
    g_sequence = torch.stack(g_sequence)
    print("g_fw_adjs size: {}".format(g_fw_adjs.size()))
    print("g_bw_adjs.size() : {}".format(g_bw_adjs.size()))
    print("num_nodes.size() : {}".format(num_nodes.size()))
    print("operation.size() : {}".format(operation.size()))
    print("operation.size() : {}".format(operation.size()))

    return {
            'num_nodes' : g_num_nodes,
            'fw_adjs': g_fw_adjs,
            'bw_adjs': g_bw_adjs,
            'operations': g_operations,
            'sequence': g_sequence
            }

In [47]:
import torch
import torch.nn as nn
import torch.nn.functional as F

"""
import tensorflow as tf
from layers import Layer, Dense
from inits import glorot, zeros
"""

class tMeanAggregator(nn.Module):
    """Aggregates via mean followed by matmul and non-linearity."""

    def __init__(self, input_dim, output_dim, neigh_input_dim=None,
            dropout=0, withBias=True, act=F.relu,
            concat=False, mode="train", **kwargs):
        super(tMeanAggregator, self).__init__(**kwargs)

        self.dropout = dropout
        self.withBias = withBias
        self.act = act
        self.concat = concat
        self.mode = mode

        
        if neigh_input_dim == None:
            neigh_input_dim = input_dim

        if concat:
            self.output_dim = 2 * output_dim

        self.neigh_weights = nn.init.xavier_uniform_(torch.empty(neigh_input_dim, output_dim))
        self.self_weights = nn.init.xavier_uniform_(torch.empty(input_dim, output_dim))
        
        if self.withBias:
            self.bias = torch.zeros(self.output_dim)

        self.input_dim = input_dim
        self.output_dim = output_dim

    def forward(self, self_vecs, neigh_vecs, neigh_len=0):
        if self.mode == "train":
            neigh_vecs = F.dropout(neigh_vecs, self.dropout)
            self_vecs = F.dropout(self_vecs, self.dropout)

        # reduce_mean performs better than mean_pool
        neigh_means = torch.mean(neigh_vecs, dim=1)
        # neigh_means = mean_pool(neigh_vecs, neigh_len)

        # [nodes] x [out_dim]
        from_neighs = torch.matmul(neigh_means, self.neigh_weights)
        from_self = torch.matmul(self_vecs, self.self_weights)

        if not self.concat:
            output = torch.add(from_self, from_neighs)
        else:
            output = torch.cat([from_self, from_neighs], dim=1)

        # bias
        if self.withBias:
            output += self.bias

        return self.act(output)

In [52]:
import model_utils.configure as conf

en = Encoder("train", conf.vocab_size, conf.hidden_layer_dim, "bi", 6, True, conf.dropout, conf.learning_rate)

i = 0
for step, sample in enumerate(queue):
    fw_adjs = sample['fw_adjs'] 
    bw_adjs = sample['bw_adjs'] 
    operations = sample['operations'] 
    num_nodes = sample['num_nodes'] 
    sequence = sample['sequence'] 

    en(fw_adjs, bw_adjs, operations, num_nodes)
    break
    i+=1
    if i == 1: break
            

enmbeded_node_rep : torch.Size([15, 16])
hop: 0, after aggregate: fw_hidden_size: torch.Size([14, 32])
hop: 1, after aggregate: fw_hidden_size: torch.Size([14, 32])
bw_hidden_size torch.Size([14, 32])
hop: 2, after aggregate: fw_hidden_size: torch.Size([14, 32])
bw_hidden_size torch.Size([14, 32])
hop: 3, after aggregate: fw_hidden_size: torch.Size([14, 32])
bw_hidden_size torch.Size([14, 32])
hop: 4, after aggregate: fw_hidden_size: torch.Size([14, 32])
bw_hidden_size torch.Size([14, 32])
hop: 5, after aggregate: fw_hidden_size: torch.Size([14, 32])
bw_hidden_size torch.Size([14, 32])
after relu hidden : torch.Size([2, 7, 64])
pooled : torch.Size([2, 64])
pooled : tensor([[0.0189, 0.1014, 0.0000, 0.0799, 0.0310, 0.0669, 0.0000, 0.1275, 0.0465,
         0.1214, 0.0595, 0.1352, 0.0496, 0.0376, 0.0006, 0.1255, 0.0000, 0.0397,
         0.0000, 0.0234, 0.0000, 0.0056, 0.0101, 0.0209, 0.0000, 0.0000, 0.0000,
         0.0448, 0.0000, 0.0706, 0.0000, 0.0000, 0.0829, 0.0521, 0.0659, 0.0000,
  

NameError: name 'grphaph_embedding' is not defined

In [6]:
# model = Graph2Seq(mode=mode, conf=conf)

# load data
# dataset = ControllerDataset(conf.data_file_path)
queue = torch.utils.data.DataLoader(dataset, batch_size=2, shuffle=True, pin_memory=False, collate_fn=collate_fn)